1. Implement a Python class MaxHeap that supports the following operations: insert, delete, and get_max. Ensure the operations maintain the properties of a max-heap.

In [2]:
class MaxHeap:
    def __init__(self):
        self.heap = []

    def insert(self, val): # Insert element function
        self.heap.append(val)
        self._sift_up(len(self.heap) - 1)

    def delete(self):   # Delete element function
        if len(self.heap) > 1:
            self._swap(0, len(self.heap) - 1)
            max_val = self.heap.pop()
            self._sift_down(0)
        elif self.heap:
            max_val = self.heap.pop()
        else:
            max_val = None
        return max_val

    def _sift_up(self, index):
        parent = (index - 1) // 2
        if index > 0 and self.heap[index] > self.heap[parent]:
            self._swap(index, parent)
            self._sift_up(parent)

    def _sift_down(self, index):
        largest = index
        left = 2 * index + 1
        right = 2 * index + 2
        if left < len(self.heap) and self.heap[left] > self.heap[largest]:
            largest = left
        if right < len(self.heap) and self.heap[right] > self.heap[largest]:
            largest = right
        if largest != index:
            self._swap(index, largest)
            self._sift_down(largest)

    def _swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def get_max(self):  # Give the max element
        if len(self.heap) == 0:
            return None
        return self.heap[0]


# Example usage: Calling all function
heap = MaxHeap()
heap.insert(10)
heap.insert(20)
heap.insert(5)
print(heap.heap)
print(heap.delete())
print(heap.heap)
print(heap.get_max())

[20, 10, 5]
20
[10, 5]
10


2. Write a Python function that takes a list of URLs, attempts to download their content, and retries up to 3 times if an error occurs. Use appropriate error handling to manage different types of exceptions.

In [4]:
import requests
from time import sleep

def download_content(urls):
    contents = []
    for url in urls:
        retries = 3
        while retries > 0:
            try:
                response = requests.get(url, timeout=5)
                response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
                contents.append(response.text)
                print(f"Successfully downloaded content from {url}")
                break  # Exit the retry loop since the request was successful
            except requests.exceptions.Timeout:
                print(f"Timeout error for {url}. Retrying... ({3 - retries + 1}/3)")
            except requests.exceptions.ConnectionError:
                print(f"Connection error for {url}. Retrying... ({3 - retries + 1}/3)")
            except requests.exceptions.HTTPError as e:
                print(f"HTTP error for {url}: {e}.")
                break  # HTTP errors are usually not retryable
            except requests.exceptions.RequestException as e:
                print(f"Error for {url}: {e}.")
                break  # Any other exceptions, break the retry loop

            retries -= 1
            sleep(1)  # Wait for 1 second before retrying

        if retries == 0:
            print(f"Failed to download content from {url} after 3 retries.")
            contents.append(None)

    return contents

# Example usage
urls = [
    "https://www.example.com",
    "https://www.nonexistentwebsite.com",
    "https://www.google.com"
]

contents = download_content(urls)
for i, content in enumerate(contents):
    if content is not None:  # Check if content is not None before slicing
        print(f"Content from URL {i+1}: {content[:100]}")
    else:
        print(f"Content from URL {i+1}: Download failed.")

Successfully downloaded content from https://www.example.com
Connection error for https://www.nonexistentwebsite.com. Retrying... (1/3)
Connection error for https://www.nonexistentwebsite.com. Retrying... (2/3)
Connection error for https://www.nonexistentwebsite.com. Retrying... (3/3)
Failed to download content from https://www.nonexistentwebsite.com after 3 retries.
Successfully downloaded content from https://www.google.com
Content from URL 1: <!doctype html>
<html>
<head>
    <title>Example Domain</title>

    <meta charset="utf-8" />
    <m
Content from URL 2: Download failed.
Content from URL 3: <!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="en"><head><meta content


3. Write a Python script that trains a simple linear regression model using scikit-learn. Use a dataset of your choice, split it into training and testing sets, and evaluate the model's performance.


In [6]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the Boston Housing dataset
california = fetch_california_housing()
data = pd.DataFrame(california.data, columns=california.feature_names)
data['PRICE'] = california.target

# Define features and target
X = data.drop('PRICE', axis=1)
y = data['PRICE']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the linear regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model's performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the results
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Optional: Print the coefficients
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coefficients)


Mean Squared Error: 0.5558915986952444
R-squared: 0.5757877060324508
            Coefficient
MedInc         0.448675
HouseAge       0.009724
AveRooms      -0.123323
AveBedrms      0.783145
Population    -0.000002
AveOccup      -0.003526
Latitude      -0.419792
Longitude     -0.433708


4. Using pandas, write a Python function to clean and preprocess a given DataFrame, which involves handling missing values, normalizing numerical columns, and encoding categorical columns.

In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

def clean_and_preprocess(df):
    # Handle missing values
    df = df.fillna(df.median(numeric_only=True))  # Fill numeric NaNs with median
    df = df.fillna(df.mode().iloc[0])  # Fill categorical NaNs with mode

    # Normalize numerical columns
    numeric_columns = df.select_dtypes(include=['number']).columns
    scaler = MinMaxScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

    # Encode categorical columns
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns
    encoder = OneHotEncoder(sparse=False, drop='first')  # Drop the first to avoid multicollinearity
    encoded_df = pd.DataFrame(encoder.fit_transform(df[categorical_columns]), columns=encoder.get_feature_names_out(categorical_columns))

    # Drop original categorical columns and concatenate the encoded columns
    df = df.drop(categorical_columns, axis=1)
    df = pd.concat([df, encoded_df], axis=1)

    return df

# Example usage
data = {
    'age': [25, 30, 35, np.nan, 40],
    'salary': [50000, 60000, 80000, 75000, np.nan],
    'gender': ['male', 'female', np.nan, 'female', 'male'],
    'department': ['sales', 'engineering', 'engineering', np.nan, 'sales']
}
df = pd.DataFrame(data)

cleaned_df = clean_and_preprocess(df)
print(cleaned_df)


        age    salary  gender_male  department_sales
0  0.000000  0.000000          1.0               1.0
1  0.333333  0.333333          0.0               0.0
2  0.666667  1.000000          0.0               0.0
3  0.500000  0.833333          0.0               0.0
4  1.000000  0.583333          1.0               1.0


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


5. Write a Python function to compute the nth Fibonacci number using recursion.


In [8]:
def fibonacci_recursive(n):
    # Base cases
    if n < 0:
        return "Error: Fibonacci sequence is not defined for negative numbers."
    elif n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fibonacci_recursive(n-1) + fibonacci_recursive(n-2)

# Get input from the user
while True:
    try:
        n = int(input("Find the nth Fibonacci number: "))
        break
    except ValueError:
        print("Invalid input. Please enter a valid integer.")

# Call the function with user input
fib_n = fibonacci_recursive(n)
print(f"The {n}th Fibonacci number is: {fib_n}")


Find the nth Fibonacci number: 10
The 10th Fibonacci number is: 55


6. Write a Python function that divides two numbers and handles the case where the divisor is zero by returning a custom error message.

In [11]:
def divide_numbers(dividend, divisor):
    try:
        result = dividend / divisor
        return result
    except ZeroDivisionError:
        return "Error: Division by zero is not allowed."

# Get input from the user
while True:
    try:
        dividend = float(input("Enter the dividend: "))
        divisor = float(input("Enter the divisor: "))
        break
    except ValueError:
        print("Invalid input. Please enter numeric values.")

# Call the function with user input
result = divide_numbers(dividend, divisor)
print(result)


Enter the dividend: 10
Enter the divisor: 0
Error: Division by zero is not allowed.


7. Write a Python decorator that measures the execution time of a function and logs it. Apply this decorator to a function that performs a computationally expensive task.


In [12]:
import time
import functools

# Decorator function to measure execution time and log it
def measure_execution_time(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        execution_time = end_time - start_time
        print(f"Function {func.__name__} executed in {execution_time:.4f} seconds")
        return result
    return wrapper

# Example: A function that performs a computationally expensive task
@measure_execution_time
def compute_factorial(n):
    factorial = 1
    for i in range(1, n+1):
        factorial *= i
    return factorial

# Example usage with user input
while True:
    try:
        n = int(input("Enter a non-negative integer to compute its factorial: "))
        if n < 0:
            print("Please enter a non-negative integer.")
        else:
            break
    except ValueError:
        print("Invalid input. Please enter a valid integer.")

result = compute_factorial(n)
print(f"The factorial of {n} is: {result}")


Enter a non-negative integer to compute its factorial: 200
Function compute_factorial executed in 0.0000 seconds
The factorial of 200 is: 788657867364790503552363213932185062295135977687173263294742533244359449963403342920304284011984623904177212138919638830257642790242637105061926624952829931113462857270763317237396988943922445621451664240254033291864131227428294853277524242407573903240321257405579568660226031904170324062351700858796178922222789623703897374720000000000000000000000000000000000000000000000000


8. Write a Python function that takes two numbers and an operator (as a string) and performs the corresponding arithmetic operation (addition, subtraction, multiplication, or division)

In [13]:
def arithmetic_operation(num1, num2, operator):
    if operator == '+':
        return num1 + num2
    elif operator == '-':
        return num1 - num2
    elif operator == '*':
        return num1 * num2
    elif operator == '/':
        # Check for division by zero
        if num2 == 0:
            return "Error: Division by zero is not allowed."
        else:
            return num1 / num2
    else:
        return f"Error: Unsupported operator '{operator}'."

# Get input from the user
while True:
    try:
        num1 = float(input("Enter the first number: "))
        num2 = float(input("Enter the second number: "))
        operator = input("Enter the operator (+, -, *, /): ")

        result = arithmetic_operation(num1, num2, operator)
        print(f"{num1} {operator} {num2} = {result}")
        break
    except ValueError:
        print("Invalid input. Please enter numeric values.")
    except KeyboardInterrupt:
        print("\n\nKeyboardInterrupt: Program terminated by user.")
        break


Enter the first number: 4
Enter the second number: 6
Enter the operator (+, -, *, /): +
4.0 + 6.0 = 10.0


9. Write a Python function that generates a random password. The password should contain a mix of uppercase letters, lowercase letters, digits, and special characters

In [14]:
import random
import string

def generate_random_password(length=12):
    # Define characters to choose from
    characters = string.ascii_letters + string.digits + string.punctuation

    # Generate password
    password = ''.join(random.choice(characters) for _ in range(length))

    return password

# Example usage:
generated_password = generate_random_password()
print(f"Generated Password: {generated_password}")


Generated Password: q9\t4QOxdI^?


10. Write a Python function that takes a 2D list (matrix) and returns its transpose

In [16]:
def transpose_matrix(matrix):
    # Check if matrix is empty
    if not matrix:
        return []

    # Dimensions of the original matrix
    rows = len(matrix)
    cols = len(matrix[0])

    # Create a new transposed matrix
    transposed = [[None]*rows for _ in range(cols)]

    # Fill the transposed matrix
    for i in range(rows):
        for j in range(cols):
            transposed[j][i] = matrix[i][j]

    return transposed

# Function to get matrix input from user
def get_matrix_input():
    try:
        rows = int(input("Enter the number of rows in the matrix: "))
        cols = int(input("Enter the number of columns in the matrix: "))

        matrix = []
        print("Enter the elements of the matrix row by row:")
        for i in range(rows):
            row = []
            for j in range(cols):
                element = int(input(f"Enter element at position ({i+1},{j+1}): "))
                row.append(element)
            matrix.append(row)

        return matrix

    except ValueError:
        print("Invalid input. Please enter valid integers.")

# Example usage with user input
matrix = get_matrix_input()
print("\nOriginal Matrix:")
for row in matrix:
    print(row)

transposed_matrix = transpose_matrix(matrix)
print("\nTransposed Matrix:")
for row in transposed_matrix:
    print(row)


Enter the number of rows in the matrix: 2
Enter the number of columns in the matrix: 3
Enter the elements of the matrix row by row:
Enter element at position (1,1): 3
Enter element at position (1,2): 4
Enter element at position (1,3): 4
Enter element at position (2,1): 2
Enter element at position (2,2): 5
Enter element at position (2,3): 2

Original Matrix:
[3, 4, 4]
[2, 5, 2]

Transposed Matrix:
[3, 2]
[4, 5]
[4, 2]
